In [ ]:
import mediapipe as mp
import cv2
import math
import pyautogui
import keyboard

In [ ]:
mp_face_detection = mp.solutions.face_detection
faces = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.6) #臉部辨識模型
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=1,
            min_detection_confidence=0.65,
            min_tracking_confidence=0.75)

In [ ]:
def vector_2d_angle(v1,v2): # 求出v1,v2兩條向量的夾角
    v1_x=v1[0]
    v1_y=v1[1]
    v2_x=v2[0]
    v2_y=v2[1]
    try:
        angle_= math.degrees(math.acos((v1_x*v2_x+v1_y*v2_y)/(((v1_x**2+v1_y**2)**0.5)*((v2_x**2+v2_y**2)**0.5))))
    except:
        angle_ = 100000.
    return angle_

def hand_angle(hand_):
    angle_list = []
    #---------------------------- thumb 大拇指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[2][0])),(int(hand_[0][1])-int(hand_[2][1]))),
        ((int(hand_[3][0])- int(hand_[4][0])),(int(hand_[3][1])- int(hand_[4][1])))
        )
    angle_list.append(angle_)
    #---------------------------- index 食指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])-int(hand_[6][0])),(int(hand_[0][1])- int(hand_[6][1]))),
        ((int(hand_[7][0])- int(hand_[8][0])),(int(hand_[7][1])- int(hand_[8][1])))
        )
    angle_list.append(angle_)
    #---------------------------- middle 中指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[10][0])),(int(hand_[0][1])- int(hand_[10][1]))),
        ((int(hand_[11][0])- int(hand_[12][0])),(int(hand_[11][1])- int(hand_[12][1])))
        )
    angle_list.append(angle_)
    #---------------------------- ring 無名指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[14][0])),(int(hand_[0][1])- int(hand_[14][1]))),
        ((int(hand_[15][0])- int(hand_[16][0])),(int(hand_[15][1])- int(hand_[16][1])))
        )
    angle_list.append(angle_)
    #---------------------------- pink 小拇指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[18][0])),(int(hand_[0][1])- int(hand_[18][1]))),
        ((int(hand_[19][0])- int(hand_[20][0])),(int(hand_[19][1])- int(hand_[20][1])))
        )
    angle_list.append(angle_)
    return angle_list
def hand_gesture(angle_list):
    gesture_str = None
    if 100000. not in angle_list:
        if (angle_list[0]>40) and (angle_list[1]<40) and (angle_list[2]>40) and (angle_list[3]>40) and (angle_list[4]>40):
            gesture_str = "1"
        elif (angle_list[0]>40) and (angle_list[1]<50) and (angle_list[2]<50) and (angle_list[3]>40) and (angle_list[4]>40):
            gesture_str = "2"
        elif (angle_list[0]>40) and (angle_list[1]<60) and (angle_list[2]<60) and (angle_list[3]<60) and (angle_list[4]>40):
            gesture_str = "3"
        elif (angle_list[0]>40) and (angle_list[1]<40) and (angle_list[2]<40) and (angle_list[3]<40) and (angle_list[4]<40):
            gesture_str = "4"
        elif (angle_list[0]>40) and (angle_list[1]>40) and (angle_list[2]>40) and (angle_list[3]>40) and (angle_list[4]>40):
            gesture_str = "0"
        elif (angle_list[0]<40) and (angle_list[1]<40) and (angle_list[2]<40) and (angle_list[3]<40) and (angle_list[4]<40):
            gesture_str = "5"
    return gesture_str

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    # BGR 2 RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Flip on horizontal
    image = cv2.flip(image, 1)

    # Set flag
    image.flags.writeable = False

    # Detections
    faces_result = faces.process(frame)
    results = hands.process(frame)
    # Set flag to true
    image.flags.writeable = True

    # RGB 2 BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Detections
    if faces_result.detections:
        #pyautogui.press('up')
        head_point = faces_result.detections[0].location_data.relative_keypoints
        # print(results)
        if head_point[0].y > head_point[1].y and abs(head_point[0].y - head_point[1].y)>0.025:
            #pyautogui.press('right')
            keyboard.press_and_release('d')
            cv2.putText(image, "Right", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        elif head_point[0].y < head_point[1].y and abs(head_point[0].y - head_point[1].y)>0.025:
            #pyautogui.press('left')
            #pyautogui.click(button='left')
            keyboard.press_and_release('a')
            cv2.putText(image, "Left", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        else:
            #print(abs(head_point[0].y - head_point[1].y))
            cv2.putText(image, "middle", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                keypoint_pos = []
                for i in range(21):
                    x = hand_landmarks.landmark[i].x*frame.shape[1]
                    y = hand_landmarks.landmark[i].y*frame.shape[0]
                    keypoint_pos.append((x,y))
                if keypoint_pos:
                    # 得到各手指的夾角資訊
                    angle_list = hand_angle(keypoint_pos)
                    # 根據角度判斷此手勢為何
                    gesture_str = hand_gesture(angle_list)
                    print(gesture_str)
                    if gesture_str == "1" :
                        pyautogui.press('1')
                    elif gesture_str == "2":
                        pyautogui.press('2')
                    elif gesture_str == "3":
                        pyautogui.press('3')
                    elif gesture_str == "4":
                        pyautogui.press('4')
                    elif gesture_str == "0":
                        keyboard.press_and_release('shift')
                    elif gesture_str == "5":
                        keyboard.press_and_release('ctrl')
    # Save our image
    # cv2.imwrite(os.path.join('Output Images', '{}.jpg'.format(uuid.uuid1())), image)
    cv2.rectangle(image, (0, 0), (355, 73), (214, 44, 53))
    cv2.putText(image, 'Direction', (15, 12),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.imshow('Hand Tracking', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()